# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [72]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,new albany,38.2856,-85.8241,15.56,87,100,4.12,US,1675919051
1,1,souillac,-20.5167,57.5167,28.21,74,75,7.20,MU,1675919052
2,2,sorong,-0.8833,131.2500,26.53,82,100,2.37,ID,1675919053
3,3,hobart,-42.8794,147.3294,21.92,62,75,4.12,AU,1675918821
4,4,saint-denis,48.9167,2.3333,-1.03,79,0,1.03,FR,1675919055


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity"
)

# Display the map
map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [64]:
# Narrow down cities that fit criteria and drop any results with null values (Made mine very
#                                                                             specific to cut down on API requests)
ideal_df = city_data_df[city_data_df['Max Temp'] <= 8]
ideal_df = ideal_df[ideal_df['Max Temp'] >= 7]

# Drop any rows with null values
# No null but coding for deliverable
ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
131,131,north bend,43.4065,-124.2243,7.70,99,0,3.09,US,1675919110
133,133,tongling,30.9500,117.7833,7.21,94,100,1.82,CN,1675919135
137,137,abnub,27.2696,31.1511,7.12,64,99,2.77,EG,1675919137
388,388,esna,25.2934,32.5540,7.01,44,100,1.97,EG,1675919171
410,410,soller,39.7662,2.7152,7.14,87,83,3.19,ES,1675919296
444,444,nalut,30.3333,10.8500,7.31,85,62,8.34,LY,1675919321
490,490,dongying,37.4564,118.4856,7.11,32,100,3.57,CN,1675919343
507,507,forio,40.7342,13.8646,7.46,71,99,5.81,IT,1675919355


### Step 3: Create a new DataFrame called `hotel_df`.

In [65]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(ideal_df.copy(deep=True))

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df[10] = ''
hotel_df = hotel_df.rename(columns={10: "Hotel Name"})
# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
131,131,north bend,43.4065,-124.2243,7.70,99,0,3.09,US,1675919110,
133,133,tongling,30.9500,117.7833,7.21,94,100,1.82,CN,1675919135,
137,137,abnub,27.2696,31.1511,7.12,64,99,2.77,EG,1675919137,
388,388,esna,25.2934,32.5540,7.01,44,100,1.97,EG,1675919171,
410,410,soller,39.7662,2.7152,7.14,87,83,3.19,ES,1675919296,
444,444,nalut,30.3333,10.8500,7.31,85,62,8.34,LY,1675919321,
490,490,dongying,37.4564,118.4856,7.11,32,100,3.57,CN,1675919343,
507,507,forio,40.7342,13.8646,7.46,71,99,5.81,IT,1675919355,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [70]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
#Tried troubleshooting to no avail. Will attempt contacting tutor for assistance.
# Display sample data
hotel_df

Starting hotel search


TypeError: 'method' object is not subscriptable

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)